In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds # to load training data


In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow_datasets as tfds
DATA_DIR="./tensorflow-datasets"


In [ ]:

tf.random.set_seed(1024)



#hyperparameter 1 batch_size = 8, batch_size = 32
#hyperparameter 2 optimizer=adam or sgd with default learning rate
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
csv_logger = tf.keras.callbacks.CSVLogger(filename="experiment_log.csv")
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])


model.summary()


In [ ]:
batch_size = 32

train_ds= tfds.load('fashion_mnist', split="train[:80%]",data_dir=DATA_DIR, shuffle_files=False).map(lambda x: (tf.cast(x['image'],tf.float32)/255.0, x['label'])).batch(batch_size)
val_ds= tfds.load('fashion_mnist', split="train[-20%:]",data_dir=DATA_DIR, shuffle_files=False).map(lambda x: (tf.cast(x['image'],tf.float32)/255.0, x['label'])).batch(batch_size)

# val_images,val_labels = tfds.load('fashion_mnist',split="train[-20%:]",shuffle_files=False,as_supervised=True).batch(batch_size)






In [ ]:
%run main.py

Working on model arch 1,batch_8,0.01,regularizer]
Epoch 1/2
6000/6000 [==============================] - 7s 1ms/step - loss: 3.0484 - accuracy: 0.5879 - val_loss: 2.8157 - val_accuracy: 0.6453
Epoch 2/2
3543/6000 [================>.............] - ETA: 2s - loss: 2.8578 - accuracy: 0.61

In [5]:
for x in ds:
    print(x[1])
    break

NameError: name 'ds' is not defined

In [ ]:
results = pd.read_csv("logs/exp1.csv")

In [ ]:
results.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
results['regularizer_yes_no'] = results['regularizer'].apply(lambda x: "Yes" if x else "No") 
sns.boxplot(x='regularizer_yes_no', y='test_accuracy', data=results)

plt.xlabel('Used regularizer?')   # Change x-axis label
plt.ylabel('Test accuracy')   # Change y-axis label
plt.title('Effects of use of L1 regularizer in test accuracy')  # Set plot title
# Show the plot
plt.savefig("l1_regularizer_test_accuracy.pdf")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='regularizer_yes_no', y='converged_on', data=results)

plt.xlabel('Used regularizer?')   # Change x-axis label
plt.ylabel('Epochs')   # Change y-axis label
plt.title('Effects of use of L1 regularizer in model convergance')  # Set plot title
plt.savefig("l1_regularizer_epoch_convergence.pdf")

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='optimizer', y='converged_on', data=results[results['regularizer']==False])

plt.xlabel('Optimizer used')   # Change x-axis label
plt.ylabel('Epochs')   # Change y-axis label
plt.title('Effects of use of optimizer in model convergance when regularizer is not used.')  # Set plot title
# plt.savefig("l1_regularizer_epoch_convergence.pdf")

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='batch_size', y='test_accuracy', data=results[results['optimizer']=='rmsprop'])

plt.xlabel('Batch size')   # Change x-axis label
plt.ylabel('Test accuracy')   # Change y-axis label
plt.title('Effects of batch size in test accuracy for RMSProp optimizer')  # Set plot title
plt.savefig("batch_size_test_accuracy_rmsprop.pdf")

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='batch_size', y='test_accuracy', data=results[results['optimizer']=='adam'])

plt.xlabel('Batch size')   # Change x-axis label
plt.ylabel('Test accuracy')   # Change y-axis label
plt.title('Effects of batch size in test accuracy for Adam optimizer')  # Set plot title
plt.savefig("batch_size_test_accuracy_adam.pdf")

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='batch_size', y='test_accuracy', data=results[results['regularizer']==False])

plt.xlabel('Batch size')   # Change x-axis label
plt.ylabel('Test accuracy')   # Change y-axis label
plt.title('Effects of batch size in test accuracy across whole experiments.')  # Set plot title
plt.savefig("batch_size_test_accuracy.pdf")

# Show the plot
plt.show()

In [ ]:
optimizers = {'adam':"adam",'rmsprop':"rmsprop"}
batch_sizes = {'batch_8':8, 'batch_32':32}
regularizers = {'regularizer':True,'no_regularizer': False}
for batch_arg,batch_size_val in batch_sizes.items():
    for optim_arg,optimizer_var in optimizers.items():
        for regularizer_arg,use_regularizer in regularizers.items():
            prefix = f"{batch_arg}_{optim_arg}_{regularizer_arg}"
            evaluation_result = pickle.load(open(f"logs/{prefix}_evaluation_results.dat","rb"))
            train_history = pd.read_csv(f"logs/{prefix}_experiment_log.csv")
            fig,axs = plt.subplots()
            axs.plot(train_history['loss'],label="Training loss")
            axs.plot(train_history['val_loss'],label="Validation loss")
            plt.title(f"Training history for batch size={batch_size_val}, optimizer={optimizer_var}, regularizer={use_regularizer}.\n Test accuracy={evaluation_result[0]} on 1000 samples")
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.legend()
            plt.savefig(f"{prefix}_training_history.pdf")
            plt.show()
            
            
            



In [ ]:
results['labels'] =results.apply(lambda x:f"batch={x['batch_size']},regularizer={x['regularizer_yes_no']},optimizer={x['optimizer']}" , axis=1)


In [ ]:
fig,ax = plt.subplots(figsize=(10,4))
sns.barplot(y='test_accuracy',x='labels',data=results,ax=ax)
plt.show()